## Introduction

In the last mission, we learned a lot about about how decorators work. In this mission, we'll continue learning more about decorators as we work with real world decorators and learn how to write decorators that take arguments.

First, let's look at some real world decorators so you can start to recognize common decorator patterns.

Memoizing is the process of storing the results of a function so that the next time the function is called with the same arguments, you can just look up the answer.

We start by setting up a dictionary that will map arguments to results. Then, as usual, we create wrapper() to be the new decorated function that this decorator returns.

In [1]:
def memoize(func):
    """Store the results of the decorated function for fast lookup
    """

    # Store results in a dict that maps arguments to results
    cache = {}

    def wrapper(*args, **kwargs):
        keys = (args, tuple(kwargs.items()))
        # If these arguments haven't been seen before, call func() and store the result.
        if keys not in cache:
            cache[keys] = func(*args, **kwargs)
        return cache[keys]

    return wrapper

When the new function gets called, we check to see whether we've ever seen these arguments before. If we haven't, we send them to the decorated function, and store the result in the cache dictionary.

Now we can look up the return value quickly in a dictionary of results. The next time we call this function with those same arguments, the return value will already be in the dictionary.

Here we are memoizing slow_function(). slow_function() simply returns the sum of its arguments. In order to simulate a slow function, we have it sleep for 5 seconds before returning.

In [2]:
import time

@memoize
def slow_function(a, b):
    print('Sleeping...')
    time.sleep(5)
    return a + b

If we call slow_function() with the arguments 3 and 4, it will sleep for 5 seconds and then return 7.

In [3]:
slow_function(3, 4)

Sleeping...


7

But if we call slow_function() with the arguments 3 and 4 again, it will immediately return 7.

In [5]:
slow_function(3, 4)

7

Because we've stored the answer in the cache, the decorated function doesn't even have to call the original slow_function() function.

You're working on a project, and you are curious about how many times each of the functions in it gets called. So you decide to write a decorator that adds a counter to each function that you decorate. You could use this information in the future to determine whether there are sections of code that you could remove because they are no longer being used. 

In [6]:
def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return func(*args, **kwargs)
        # Call the function being decorated and return the result

    wrapper.count = 0
    # Return the new decorated function
    return wrapper

@counter
def foo():
    print('calling foo()')    

foo()
foo()

print('foo() was called {} times.'.format(foo.count))

calling foo()
calling foo()
foo() was called 2 times.


### Real World decorators

Let's look at another example of a real world decorator.

The `timer()` decorator runs the decorated function and then prints how long it took for the function to run. It's good to add some version of this to all of your projects because it is a pretty easy way to figure out where your computational bottlenecks are.

In [ ]:
import time

def timer(func):
    """A decorator that prints how long a function took to run.

    Args:
         func (callable): The function being decorated.

    Returns:
         callable: The decorated function.
    """

All decorators have fairly similar looking docstrings because they all take and return a single function. For brevity, we will only include the description of the function in the docstrings of the examples that follow.

Like most decorators, we'll start off by defining a wrapper() function. This is the function that the decorator will return. wrapper() takes any number of positional and keyword arguments, so that it can be used to decorate any function.

In [7]:
def timer(func):
    """A decorator that prints how long a function took to run."""

    # Define the wrapper function to return.
    def wrapper(*args, **kwargs):
        # When wrapper() is called, get the current time.
        t_start = time.time()

        # Call the decorated function and store the result.
        result = func(*args, **kwargs)

        # Get the total time it took to run, and print it.
        t_total = time.time() - t_start

        print('{} took {}s'.format(func.__name__, t_total))        
        return result

    return wrapper

The first thing the new function will do is record the time that it was called with the time() function. Then wrapper() gets the result of calling the decorated function. We don't return that value yet, though.

After calling the decorated function, wrapper() checks the time again, and prints a message about how long it took to run the decorated function.

Once we've done that, we need to return the value that the decorated function calculated.

So if we decorate this simple sleep_n_seconds() function, you can see that sleeping for 5 seconds takes about 5 seconds and sleeping for 10 seconds takes about 10 seconds.

In [8]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)

In [9]:
sleep_n_seconds(5)

sleep_n_seconds took 5.004017114639282s


In [10]:
sleep_n_seconds(10)

sleep_n_seconds took 10.009540319442749s


This is a trivial use of the decorator to show it working, but it can be very useful for finding the slow parts of your code.

**Exercise**

Write a decorator, print_return_type(), that will print out the type of the variable that gets returned from every call of any function it is decorating.

In [12]:
def print_return_type(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(func.__name__, type(result)))
        return result
    return wrapper


@print_return_type
def foo(value):
    return value

print(foo(42))
print(foo([1,2,3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


### Preserving metadata when decorating functions

One of the problems with decorators is that they obscure the decorated function's metadata.

Let's return to the sleep_n_seconds() function from the previous screen and add a docstring that explains exactly what it does.

In [13]:
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.

    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

If we look at the docstring attribute, we can see the text of the docstring.

In [15]:
print(sleep_n_seconds.__doc__)

Pause processing for n seconds.

    Args:
        n (int): The number of seconds to pause for.
    


We can also access other metadata for the function, like its name and default arguments.

In [17]:
print(sleep_n_seconds.__name__)

sleep_n_seconds


In [18]:
print(sleep_n_seconds.__defaults__)

(10,)


Next, let's see what happens when we decorate sleep_n_seconds() with the timer() decorator and try to access its metadata again.

In [20]:
def timer(func):
    """A decorator that prints how long a function took to run."""  
    def wrapper(*args, **kwargs):
        t_start = time.time()
    
        result = func(*args, **kwargs)
    
        t_total = time.time() - t_start
        print('{} took {}s'.format(func.__name__, t_total))
    
        return result
    return wrapper

@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.
  
    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)
    

print(sleep_n_seconds.__doc__)
print(sleep_n_seconds.__name__)

None
wrapper


Remember that when we write decorators we almost always define a nested function to return. Because the decorator overwrites the sleep_n_seconds() function, when we ask for sleep_n_seconds()'s docstring or name, we are actually referencing the nested function that was returned by the decorator. In this case, the nested function was called wrapper() and it didn't have a docstring.

Fortunately, Python provides us with an easy way to fix this. The `wraps()` function from the functools module is a decorator that we use when defining a decorator. If we use it to decorate the wrapper function that our decorator returns, it will modify wrapper()'s metadata to look like the function we are decorating.

In [21]:
from functools import wraps

def timer(func):
    """A decorator that prints how long a function took to run."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        t_total = time.time() - t_start
        
        print('{} took {}s'.format(func.__name__, t_total))

        return result
    return wrapper

Notice that the wraps() decorator takes the function we are decorating as an argument. We haven't talked about **decorators that take arguments yet**, but we will cover that in the next screen.

If we use this updated version of the timer() decorator to decorate sleep_n_seconds() and then try to print sleep_n_seconds()'s docstring, we get the result we expect.

In [22]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.

    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

In [23]:
print(sleep_n_seconds.__doc__)

Pause processing for n seconds.

    Args:
        n (int): The number of seconds to pause for.
    


Likewise, printing the name or any other metadata now gives us the metadata from the function being decorated rather than the metadata of the wrapper() function.

As an added bonus, using wraps() when creating our decorator also gives us easy access to the original undecorated function via the `__wrapped__` attribute. Of course, we always had access to this function via the closure, but this is an easy way to get to it if we need it.

In [24]:
@timer
def sleep_n_seconds(n=10):
    """Pause processing for n seconds.

    Args:
        n (int): The number of seconds to pause for.
    """
    time.sleep(n)

In [25]:
sleep_n_seconds.__wrapped__

<function __main__.sleep_n_seconds(n=10)>

### Adding arguments to decorators

Sometimes it would be nice to add arguments to our decorators. To do that, we need another level of nesting in our decorators.

Let's consider this silly `run_three_times()` decorator. If you use it to decorate a function, it will run that function three times.

In [26]:
def run_three_times(func):
    def wrapper(*args, **kwargs):
        for i in range(3):
            func(*args, **kwargs)
    return wrapper

If we use it to decorate the print_sum() function and then run print_sum(3,5), it will print 8 three times.

In [27]:
@run_three_times
def print_sum(a, b):
    print(a + b)

print_sum(3, 5)

8
8
8


Let's think about what we would need to change if we wanted to write a `run_n_times()` decorator. We want to pass `n` in as an argument, instead of hard-coding in the decorator.

In [ ]:
def run_n_times(func):
    def wrapper(*args, **kwargs):
        # How do we pass "n" into this function?
        for i in range(???):
            func(*args, **kwargs)
    return wrapper

If we had some way to pass n into the decorator, we could decorate `print_sum()` so that it gets run three times and decorate the `print_hello()` function to run five times.

In [ ]:
@run_n_times(3)
def print_sum(a, b):
    print(a + b)

In [ ]:
@run_n_times(5)
def print_hello():
    print('Hello!')

But a decorator is only supposed to take one argument - the function it is decorating. Also, when we use decorator syntax, we're not supposed to use the parentheses. So how can this be done?

To make run_n_times() work, we have to **turn it into a function that returns a decorator, rather than a function that is a decorator.**

Let's turn run_n_times() into a function that returns a decorator so we can add arguments to it.

We'll start by redefining `run_n_times()` so that it takes n as an argument, instead of `func`. Then, inside of `run_n_times()`, we'll define a new decorator function. This function takes `func` as an argument, because it is the function that will be acting as our decorator.

In [29]:
def run_n_times(n):
    """Define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

We start our new decorator with a nested `wrapper()` function, as usual.

Now, since we are still inside the `run_n_times()` function, we have access to the n parameter that was passed to run_n_times(). We can use that to control how many times we repeat the loop that calls our decorated function.

As usual for any decorator, we return the new `wrapper()` function. And, if `run_n_times()` returns the decorator() function we just defined, then we can use that return value as a decorator.

Notice how when we decorate print_sum() with run_n_times(), we use parentheses after @run_n_times.

In [33]:
@run_n_times(3)
def print_sum(a, b):
    print(a + b)

This indicates that we are actually calling run_n_times() and decorating print_sum() with the result of that function call. Since the return value from run_n_times() is a decorator function, we can use it to decorate print_sum().

When we use decorator syntax, the thing that comes after the @ symbol must be a reference to a decorator function. We can use the name of a specific decorator, or we can call a function that returns a decorator.


In [32]:
print_sum(1,2)

3
3
3


### Real world decorators with arguments

Let's finish up by looking at an example of a real world decorator that takes an argument to get a better sense of how they work.

For our first example, let's imagine that we have some functions that occasionally either run for longer than we want them to, or just hang and never return.

It would be nice if we could add some kind of timeout() decorator to those functions that will raise an error if the function runs for longer than expected.

```
@timeout
def function1():
    # This function sometimes
    # runs for a loooong time
    ...
```

```
@timeout
def function2():
    # This function sometimes
    # hangs and doesn't return
    ...
```

To create the `timeout()` decorator, we are going to use some functions from Python's `signal` module. These functions have nothing to do with decorators, but understanding them will help us understand the timeout() decorator.

In [38]:
import signal

The raise_timeout() function simply raises a TimeoutError when it is called.

In [35]:
def raise_timeout(*args, **kwargs):
    raise TimeoutError()

The `signal()` function tells Python, *"When you see the signal whose number is signalnum, call the handler function."* In this case, we tell Python to call raise_timeout() whenever it sees the alarm signal.

In [ ]:
# When an "alarm" signal goes off, call raise_timeout()
signal.signal(signalnum=signal.SIGALRM, handler=raise_timeout)

The alarm() function lets us set an alarm for some number of seconds in the future.

In [ ]:
# Set off an alarm in 5 seconds
signal.alarm(5)

Passing 0 to the alarm() function cancels the alarm.

In [ ]:
# Cancel the alarm
signal.alarm(0)

We'll start by creating a decorator that times out in exactly 5 seconds, and then build from there to create a decorator that takes the timeout length as an argument.

Our timeout_in_5s() decorator starts off by defining a wrapper() function to return as the new decorated function. Returning this function is what makes timeout_in_5s() a decorator.

In [40]:
def timeout_in_5s(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Set an alarm for 5 seconds
        signal.alarm(5)
        try:
            # Call the decorated func
            return func(*args, **kwargs)
        finally:
            # Cancel alarm
            signal.alarm(0)
    return wrapper

First wrapper() sets an alarm for 5 seconds in the future.

Then it calls the function being decorated. It wraps that call in a try block so that in a finally block we can cancel the alarm. This ensures that the alarm either rings or gets canceled. Remember, when the alarm rings, Python calls the raise_timeout() function.

Let's use timeout_in_5s() to decorate a function that will definitely timeout next. foo() sleeps for 10 seconds and the prints "foo!".

In [41]:
@timeout_in_5s
def foo():
    time.sleep(10)
    print('foo!')

If we call foo(), the 5 second alarm will ring before it finishes sleeping, and Python will raise a TimeoutErrror.

In [ ]:
foo()   # -> Timeout Error

Next, let's create a more useful version of the timeout() decorator that takes the timeout length as an argument. In order to do so, we'll change the timeout_in_5s decorator to a function that returns a decorator. So, when we call timeout(), we want it to return a brand new decorator that times out in 5 seconds, or 20 seconds, or whatever value we pass to it.

In [ ]:
import signal

def timeout(n_seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set an alarm for 5 seconds
            signal.alarm(n_seconds)
            try:
                # Call the decorated func
                return func(*args, **kwargs)
            finally:
                # Cancel alarm
                signal.alarm(0)
        return wrapper
    return decorator

@timeout(20)
def bar():
    time.sleep(10)
    print('bar!')
    
bar()

Another example for tagging:

In [ ]:
def tag(*tags):
    # Define a new decorator to return
    def decorator(func):
        # Ensure the decorated function keeps its metadata
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Call the function being decorated and return the result
            return func(*args, **kwargs)
        wrapper.tags = tags
        return wrapper
    # Return the new decorator
    return decorator


@tag('test', 'this is a tag')
def foo():
    pass

print(foo.tags)